In [15]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
%matplotlib inline 
import numpy as np
plt.rcParams['figure.figsize'] = (16, 4)

import warnings
warnings.filterwarnings(action="ignore")
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

#from utils import * # We've added all the functions from the last BLU to the utils.py 

In [16]:
import pmdarima as pm
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer

In [17]:
wf1_df = pd.read_csv('data/wf1_data.csv')  
wf2_df = pd.read_csv('data/wf2_data.csv')  
wf3_df = pd.read_csv('data/wf3_data.csv')

In [18]:
wf1_df['date'] = pd.to_datetime(wf1_df.date, format='%Y%m%d%H')
wf2_df['date'] = pd.to_datetime(wf2_df.date, format='%Y%m%d%H')
wf3_df['date'] = pd.to_datetime(wf3_df.date, format='%Y%m%d%H')

In [19]:
wf1_df = wf1_df.set_index('date',    # <---- Set the index to be our timestamp data  
                      drop=True)
wf2_df = wf2_df.set_index('date',    # <---- Set the index to be our timestamp data  
                      drop=True)
wf3_df = wf3_df.set_index('date',    # <---- Set the index to be our timestamp data  
                      drop=True)

In [20]:
data_wf1 = wf1_df[:-24]
vtp_wf1 = wf1_df[-24:]

data_wf2 = wf2_df[:-24]
vtp_wf2 = wf2_df[-24:]

data_wf3 = wf3_df[:-24]
vtp_wf3 = wf3_df[-24:]

In [21]:
len(wf1_df)

8759

In [23]:
#data_limit = 720  #30 days * 24h
data_limit = 2160 #30 days * 24h * 3 months

In [24]:
data_wf1_r = data_wf1.resample('1H').mean()
missing_values1 = data_wf1_r[data_wf1_r.isnull()['wp']]

In [25]:
from sklearn.impute import KNNImputer
imputer = KNNImputer() # To obtain better results we may want to have more neighbors, of course that in a real setting
                       # this is a parameter that should be tunned
    
data_imputed1 = imputer.fit_transform(data_wf1_r) # This may take a bit to run
data_wf3_imputed = pd.DataFrame(data_imputed1, index=data_wf1_r.index, columns=data_wf1_r.columns)
data_wf1_imputed

NameError: name 'data_wf1_imputed' is not defined

In [ ]:
def build_some_features(df_, num_periods_lagged=1, num_periods_diffed=0, weekday=False, month=False, rolling=[], holidays=False):
    """
    Builds some features by calculating differences between periods
    """
    # make a copy
    df_ = df_.copy()
    # for a few values, get the lags
    for i in range(1, num_periods_lagged+1):
        # make a new feature, with the lags in the observed values column
        df_['lagged_%s' % str(i)] = df_['customers'].shift(i)
    # for a few values, get the diffs
    for i in range(1, num_periods_diffed+1):
        # make a new feature, with the lags in the observed values column
        df_['diff_%s' % str(i)] = df_['customers'].diff(i)
    for stat in rolling:
        df_['rolling_%s'%str(stat)] = df_['customers'].rolling('7D').aggregate(stat)
    if weekday == True:
        df_['sin_weekday'] = np.sin(2*np.pi*df_.index.weekday/7)
        df_['cos_weekday'] = np.cos(2*np.pi*df_.index.weekday/7)
    if month == True:
        df_['sin_month'] = np.sin(2*np.pi*df_.index.month/12)
        df_['cos_month'] = np.cos(2*np.pi*df_.index.month/12)
    if hour == True:
        df_['sin_hour'] = np.sin(2*np.pi*df_.index.hour/12)
        df_['cos_hour'] = np.cos(2*np.pi*df_.index.hour/12)
        
    if holidays == True:
        holidays = df_[((df_.index.month==12) & (df_.index.day==25))
              |((df_.index.month==1) & (df_.index.day==1))].customers
        df_['holidays'] = holidays + 1
        df_['holidays'] = df_['holidays'].fillna(0)
    return df_

In [26]:
def predict_period_n(series_, model, number_of_periods_ahead, num_periods_lagged, num_periods_diffed, weekday, month, rolling, holidays): 
    
        X_train, y_train, X_last_period = prepare_for_prediction(series_, 
                                                             number_of_periods_ahead, 
                                                             num_periods_lagged,
                                                             num_periods_diffed,
                                                             weekday,
                                                             month,
                                                             rolling,
                                                             holidays)
        
        model.fit(X_train, y_train)
        return model.predict(X_last_period.values.reshape(1, -1))

In [27]:
from sklearn.model_selection import ParameterGrid
param_grid = {'model': [ GradientBoostingRegressor()],
              'num_periods_lagged':np.arange(1,3),
              'num_periods_diffed':[0],
              'weekday':[True,False],
              'month':[True,False],
              'holidays': [True],
              'rolling' : [[np.mean,np.min,np.max,np.std]]
             }
grid = ParameterGrid(param_grid)

###Seperate train test and validation
error_lst = []
for params in tqdm(grid):
    predictions = predict_n_periods(series_=train_wp,
                    n_periods=24,
                      model=params['model'],
                      num_periods_lagged=params['num_periods_lagged'],
                      num_periods_diffed=params['num_periods_diffed'],
                      weekday=params['weekday'],
                      month=params['month'],
                      rolling=[np.mean,np.max,np.min]
                    )
    error_lst.append(mean_absolute_error(val,predictions))
pd.Series(error_lst).idxmin()

NameError: name 'predict_n_periods' is not defined

In [29]:
data_wf1_imputed = build_some_features(series_=data_wf1_imputed,)
data_wf1_imputed['day'] = data_wf1_imputed.index.day
data_wf1_imputed['month'] = data_wf1_imputed.index.month
data_wf1_imputed['hour'] = data_wf1_imputed.index.hour
data_wf1_imputed.head()

NameError: name 'data_wf1_imputed' is not defined

In [30]:
data_wf1_imputed = data_wf1_imputed.sort_index()
data1 = data_wf1_imputed
ws1 = data1.drop(['wp'],axis = 1)
data1 = data1['wp'] #data3.drop(['ws'],axis = 1)
#data3.plot()

NameError: name 'data_wf1_imputed' is not defined

In [40]:
data3_c = data3
ws3_c = ws3

In [41]:
#data3_s = data3_c[-data_limit:]
#ws3_s = ws3_c[-data_limit:]
data3_s = data3_c
ws3_s = ws3_c

In [42]:
data3_t_s = data3_s[:-24]
data3_test_s = data3_s[-24:]

data3_val_s = data3_t_s[-24:]
data3_train_s = data3_t_s[:-24]
 
ws3_t_s = ws3_s[:-24]
ws3_test_s = ws3_s[-24:]

ws3_val_s = ws3_t_s[-24:]
ws3_train_s = ws3_t_s[:-24]

In [ ]:
sarimax_pipeline_exog = Pipeline([  #The boxcox transform has been removed because it messes with the exogenous input
    ('arima', pm.AutoARIMA(trace=True,
                           suppress_warnings=True,
                           m=24, 
                           method='nm',
                           maxiter=20,
                           
                          )
    )
])

sarimax_pipeline_exog.fit(data3_t_s,ws3_t_s)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[24] intercept   : AIC=-14350.560, Time=15.88 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=-13766.703, Time=1.56 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=-13906.853, Time=18.25 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=-13938.778, Time=18.89 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=4520.395, Time=1.28 sec
 ARIMA(2,1,2)(0,0,1)[24] intercept   : AIC=-14355.925, Time=22.22 sec
 ARIMA(2,1,2)(0,0,0)[24] intercept   : AIC=-14354.078, Time=2.62 sec
 ARIMA(2,1,2)(0,0,2)[24] intercept   : AIC=-14356.402, Time=83.15 sec
 ARIMA(2,1,2)(1,0,2)[24] intercept   : AIC=-14353.240, Time=73.65 sec
 ARIMA(1,1,2)(0,0,2)[24] intercept   : AIC=-14397.517, Time=68.52 sec
 ARIMA(1,1,2)(0,0,1)[24] intercept   : AIC=-14394.664, Time=18.40 sec
 ARIMA(1,1,2)(1,0,2)[24] intercept   : AIC=-14395.997, Time=75.87 sec
 ARIMA(1,1,2)(1,0,1)[24] intercept   : AIC=-14392.400, Time=16.70 sec
 ARIMA(0,1,2)(0,0,2)[24] intercept   : AIC=-14030.14

In [ ]:
wf3_predict = sarimax_pipeline_exog.predict(24,ws3_test_s)

In [ ]:
plt.plot(data3_test_s, label='original')
plt.plot(data3_test_s.index, wf3_predict, label='sarimax_exog')
plt.legend()

In [ ]:
data_wf2_r = data_wf2.resample('1H').mean()
missing_values2 = data_wf2_r[data_wf2_r.isnull()['wp']]

imputer = KNNImputer() # To obtain better results we may want to have more neighbors, of course that in a real setting
                       # this is a parameter that should be tunned
    
data_imputed2 = imputer.fit_transform(data_wf2_r) # This may take a bit to run
data_wf2_imputed = pd.DataFrame(data_imputed2, index=data_wf2_r.index, columns=data_wf2_r.columns)
data_wf2_imputed = data_wf2_imputed.sort_index()

data_wf2_imputed['day'] = data_wf2_imputed.index.day
data_wf2_imputed['month'] = data_wf2_imputed.index.month
data_wf2_imputed['hour'] = data_wf2_imputed.index.hour


data2 = data_wf2_imputed
ws2 = data2['ws']
#ws2 = data2.drop(['wp'],axis = 1)
data2 = data2.drop(['ws'],axis = 1)
data2_c = data2
ws2_c = ws2
#data2_s = data2_c[-data_limit:]
#ws2_s = ws2_c[-data_limit:]
data2_s = data2_c
ws2_s = ws2_c
data2_t_s = data2_s[:-24]
data2_test_s = data2_s[-24:]

data2_val_s = data2_t_s[-24:]
data2_train_s = data2_t_s[:-24]
 
ws2_t_s = ws2_s[:-24]
ws2_test_s = ws2_s[-24:]

ws2_val_s = ws2_t_s[-24:]
ws2_train_s = ws2_t_s[:-24]

sarimax_pipeline_exog = Pipeline([  #The boxcox transform has been removed because it messes with the exogenous input
    ('arima', pm.AutoARIMA(trace=True,
                           suppress_warnings=True,
                           m=24, 
                           method='nm',
                           maxiter=20,
                           
                          )
    )
])

sarimax_pipeline_exog.fit(data2_t_s,ws2_t_s)

In [ ]:
wf2_predict = sarimax_pipeline_exog.predict(24,ws2_test_s)
plt.plot(data2_test_s, label='original')
plt.plot(data2_test_s.index, wf2_predict, label='sarimax_exog')
plt.legend()

In [ ]:
#####################
#predictions = np.exp(sarimax.predict_in_sample())
mae = mean_absolute_error(wf2_predict,emissions_train)

print(mae)
#####################

In [ ]:
data_wf1_r = data_wf1.resample('1H').mean()
missing_values1 = data_wf1_r[data_wf1_r.isnull()['wp']]

imputer = KNNImputer() # To obtain better results we may want to have more neighbors, of course that in a real setting
                       # this is a parameter that should be tunned
    
data_imputed1 = imputer.fit_transform(data_wf1_r) # This may take a bit to run
data_wf1_imputed = pd.DataFrame(data_imputed1, index=data_wf1_r.index, columns=data_wf1_r.columns)
data_wf1_imputed = data_wf1_imputed.sort_index()

data_wf1_imputed['day'] = data_wf1_imputed.index.day
data_wf1_imputed['month'] = data_wf1_imputed.index.month
data_wf1_imputed['hour'] = data_wf1_imputed.index.hour

data1 = data_wf1_imputed
#ws1 = data1.drop(['wp'],axis = 1)
ws1 = data1['ws']
data1 = data1.drop(['ws'],axis = 1)
data1_c = data1
ws1_c = ws1
data1_s = data1_c[-data_limit:]
ws1_s = ws1_c[-data_limit:]
data1_t_s = data1_s[:-24]
data1_test_s = data1_s[-24:]

data1_val_s = data1_t_s[-24:]
data1_train_s = data1_t_s[:-24]
 
ws1_t_s = ws1_s[:-24]
ws1_test_s = ws1_s[-24:]

ws1_val_s = ws1_t_s[-24:]
ws1_train_s = ws1_t_s[:-24]

sarimax_pipeline_exog = Pipeline([  #The boxcox transform has been removed because it messes with the exogenous input
    ('arima', pm.AutoARIMA(trace=True,
                           suppress_warnings=True,
                           m=24, 
                           method='nm',
                           maxiter=20,
                           
                          )
    )
])

sarimax_pipeline_exog.fit(data1_t_s,ws1_t_s)

In [ ]:
wf1_predict = sarimax_pipeline_exog.predict(24,ws1_test_s)
plt.plot(data1_test_s, label='original')
plt.plot(data1_test_s.index, wf1_predict, label='sarimax_exog')
plt.legend()

In [ ]:
#generate_submission_file(wf1_predict , wf2_predict , wf3_predict)